<img src="header.png" align="left"/>

# Exercise Import of excel data and timeseries prediction

The goal of this example is to show the work with time series from Excel files and to develop a prediction model for time series. The data set describes the development of passenger numbers of an airline in the distant past. However, the data is typical for data as found in SME's (e.g. sales figures). 


The code for this example was adapted from [1]. Further information is available here [2]. 

- [1] [https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
- [2] [https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa](https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa)


Citation dataset:
```
Box, G. E. P., Jenkins, G. M. and Reinsel, G. C. (1976) Time Series Analysis, Forecasting and Control. Third Edition. Holden-Day. Series G.
```


# Import of modules

In [ ]:
import os
import openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

plt.rcParams["figure.figsize"] = (16,9)

In [ ]:
#
# handle error messages
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)

In [ ]:
#
# Für GPU Support
#
import tensorflow as tf
print ( tf.__version__ ) 

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR )
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Constants

In [ ]:
excelData = 'data/airline_passengers.xlsx'

# Read the data from Excel file

In [ ]:
data = pd.ExcelFile(excelData)
print(data.sheet_names)

In [ ]:
#
# read one table
#
df = data.parse('Tabellenblatt1')
df.info
df.head(10)

In [ ]:
df.tail()

# Cut out data from pandas dataframes

Tutorial: [https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)

<img src="info.png" align="left"/> 

In [ ]:
#
# cut
#
df_cut = df.iloc[3:146,1:3]

In [ ]:
df_cut.head()

In [ ]:
df_cut.columns = ['month','passengers']

In [ ]:
df_cut.head()

In [ ]:
df_cut['passengers'].plot()

# Conventional methods

There are many older methods to work with time series:

- https://machinelearningmastery.com/time-series-trends-in-python/
- https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa


The main purpose of these methods is the calculation and use of parameters such as **trend** and **seasonality**. We hope that our models can handle them without us having to explicitly deal with them.

<img src="info.png" align="left"/> 

In [ ]:
x_data = df_cut['passengers'].values
x_data = x_data.astype('float32')
x_data = np.reshape(x_data,(-1,1))
print(x_data)

In [ ]:
#
# Normalize the dataset
#
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
scaler.fit(x_data)

In [ ]:
print(scaler.data_max_)

In [ ]:
x_data = scaler.transform(x_data)
print(x_data)

In [ ]:
# split into train and test sets
train_size = int(len(x_data) * 0.80)
test_size = len(x_data) - train_size
train, test = x_data[0:train_size,:], x_data[train_size:len(x_data),:]
print(len(train), len(test))

# Create a training data set with sliding windows

Sliding windows are created from a time series by dragging a window over the entire time series and copying the data in the window. More details here [https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe](https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe).

<img src="info.png" align="left"/> 

In [ ]:
#
# create sliding windows with one label (y)
#
def createSlidingWindowsWithLabel(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[ i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
#
# apply sliding window function with window size (look_back)
#
look_back = 1
trainX, trainY = createSlidingWindowsWithLabel(train, look_back)
testX, testY = createSlidingWindowsWithLabel(test, look_back)

In [ ]:
# 
# reshape input to be [samples, time steps, features]
#
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
#
# create and fit the LSTM network
#
def createLSTMModel():
    model = Sequential()
    model.add(LSTM(6, input_shape=(1, look_back)))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
lstm_model = createLSTMModel()

In [ ]:
history = lstm_model.fit(trainX, trainY, epochs=60, batch_size=4, verbose=1, validation_data=(testX, testY), shuffle=False)

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
#
# Schätzung der Werte für train und test Daten
#
trainPredict = lstm_model.predict(trainX)
testPredict = lstm_model.predict(testX)

In [ ]:
#
# Rücktransformation der Schätzungen (scaler)
#
trainPredict = scaler.inverse_transform(trainPredict)
trainYi = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testYi = scaler.inverse_transform([testY])

In [ ]:
#
# Anzeige der geschätzten neuen Werte
#
plt.plot(testYi[0,0:])
plt.plot(testPredict[1:,0])
plt.show()

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainYi[0,0:-1], trainPredict[1:,0]))
print('train loss: %.3f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testYi[0,0:-1], testPredict[1:,0]))
print('test loss: %.3f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[:len(trainPredict)+0, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(x_data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2):len(x_data)-2, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.plot(scaler.inverse_transform(x_data), color='grey')
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

# Experiment with additional features

**idea**: There is more information in this record than we have used so far. For example the month of a year as well as the year. We can add this information to the data set to give the model more information. We can derive this additional information from the date feature in the original data.

In [1]:
df_cut.head()

NameError: name 'df_cut' is not defined

In [ ]:
df_cut['date'] = pd.to_datetime(df_cut['month'], infer_datetime_format=True )

In [ ]:
df_cut.head()

In [ ]:
df_cut['year'] = pd.DatetimeIndex(df_cut['date']).year
df_cut['month'] = pd.DatetimeIndex(df_cut['date']).month

In [ ]:
df_cut.head()

In [ ]:
x_data_raw = df_cut.loc[:, ['month', 'passengers','year']].values
print(x_data_raw)

In [ ]:
x_data_fl = x_data_raw.astype('float32')

In [ ]:
x_data_mv_raw = np.reshape(x_data_fl,(-1,3))

In [ ]:
print(x_data_mv_raw)

In [ ]:
#
# Normalize the dataset
#
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
scaler.fit(x_data_mv_raw)

In [ ]:
x_data_mv_scaled = scaler.transform(x_data_mv_raw)
print(x_data_mv_scaled)

In [ ]:
plt.plot(x_data_mv_scaled)

In [ ]:
#
# transformation of a multivariate timeseries with sliding windows and one feature (y)
#
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('x%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('y%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('y%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
#
# Generate
#
windowSize = 12
x_data_mv_windowed = series_to_supervised(x_data_mv_scaled, windowSize, 1)
print(x_data_mv_windowed.head())

In [ ]:
#
# delete labes which dont need to be estimated (month, year)
#
x_data_mv_windowed.drop(x_data_mv_windowed.columns[[windowSize*3,(windowSize*3)+2]], axis=1, inplace=True)
print(x_data_mv_windowed.head())

In [ ]:
#x_data_windowed.plot()

In [ ]:
# split into train and test sets
x_data_mv = x_data_mv_windowed.values
train_size_mv = int(len(x_data) * 0.70)
test_size_mv = len(x_data) - train_size_mv
train_mv, test_mv = x_data_mv[0:train_size_mv,:], x_data_mv[train_size_mv:len(x_data),:]
print(train_mv.shape, test_mv.shape)

In [ ]:
#
# separate features und labels
#
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [ ]:
#
# reshape for model
#
train_X_r = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X_r = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X_r.shape, train_y.shape, test_X_r.shape, test_y.shape)

In [ ]:
#
# create and fit the LSTM network
#
def createMVLSTMModel():
    model = Sequential()
    model.add(LSTM(20, dropout=0.2, input_shape=(train_X_r.shape[1], train_X_r.shape[2])))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
model = createMVLSTMModel()
history = model.fit(train_X_r, train_y, epochs=300, batch_size=24, validation_data=(test_X_r, test_y), verbose=1, shuffle=False)    

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
#
# Berechung der Vorhersagen
#
yhat = model.predict(test_X_r,batch_size=1)
trainyhat = model.predict(train_X_r,batch_size=1)

In [ ]:
def invertForecastScaling(x,y):
    # invert scaling for forecast
    inv_yhat = np.concatenate( ( x[:,0:1], y[:,:], x[:, 0:1] ), axis=1 )
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,1]
    return inv_yhat

In [ ]:
inv_yhat = invertForecastScaling(test_X,yhat)
print(inv_yhat)

In [ ]:
inv_y = invertForecastScaling(test_X,test_y.reshape(-1,1))
print(inv_y)

In [ ]:
plt.plot(inv_yhat,color='b')
plt.plot(inv_y,color='g')
plt.show()

In [ ]:
# calculate RMSE
testScore = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test loss: %.3f RMSE' % testScore)

In [ ]:
inv_y_train = invertForecastScaling(train_X,trainyhat.reshape(-1,1))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data_raw[:,1])
trainPredictPlot[:] = np.nan
trainPredictPlot[windowSize:len(inv_y_train)+windowSize] = inv_y_train

In [ ]:
# shift test predictions for plotting
testPredictPlot = np.empty_like(x_data_raw[:,1])
testPredictPlot[:] = np.nan
testPredictPlot[len(x_data_raw)-len(inv_yhat):len(x_data_raw)] = inv_yhat

In [ ]:
# plot baseline and predictions
plt.plot(x_data_raw[:,1])
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()